In [ ]:
import torch, os
import pandas as pd
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast
from torch.utils.data import Dataset
from google.colab import drive
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
drive.mount('/content/drive')

# Set the file path to the Google Drive file
file_path = "/thesis_dataset.csv"

df_org = pd.read_csv(file_path)

# Convert all labels to lowercase and strip any leading/trailing whitespace
df_org['Labels'] = df_org['Labels'].str.lower().str.strip()

# Replace 'no' with 'no depression'
df_org['Labels'] = df_org['Labels'].replace('no', 'no depression')

df_org.dropna(inplace=True)

# Shuffle the DataFrame
df_org = df_org.sample(frac=1.0, random_state=42)


# Display the first few rows
df_org.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Tweets,Labels
6003,"i have a healthy head of hair, boyish good loo...",atypical
9482,dunlap mil sack amp lawson mil sacks he isn pl...,no depression
7625,still laughing abt dude saying nvm being silly...,no depression
4692,@mma_like I think you might have the Kanye bip...,bipolar
8894,because of the eras if he played vs people suc...,no depression


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
labels = df_org['Labels'].unique().tolist()
labels = [s.strip() if isinstance(s, str) else s for s in labels]
print(labels)

['atypical', 'no depression', 'bipolar', 'postpartum', 'major depressive', 'psychotic']


In [ ]:
for key, value in enumerate(labels):
    print(value)

atypical
no depression
bipolar
postpartum
major depressive
psychotic


In [ ]:
# Original labels
labels = ['atypical', 'no depression', 'bipolar', 'postpartum', 'major depressive', 'psychotic']

# Number of labels
NUM_LABELS = len(labels)

# Create initial ID to label mapping
id2label = {id: label for id, label in enumerate(labels)}

# Create initial label to ID mapping
label2id = {label: id for id, label in enumerate(labels)}

# Sort labels and ensure 'no depression' is at index 0
sorted_labels = sorted(label2id.keys())
if 'no depression' in sorted_labels:
    sorted_labels.remove('no depression')
    sorted_labels.insert(0, 'no depression')

# Rebuild label to ID mapping
label2id = {label: idx for idx, label in enumerate(sorted_labels)}

# Rebuild ID to label mapping to reflect changes
id2label = {idx: label for idx, label in enumerate(sorted_labels)}

# Print the mappings
print("NUM_LABELS:", NUM_LABELS)
print("label2id:", label2id)
print("id2label:", id2label)


NUM_LABELS: 6
label2id: {'no depression': 0, 'atypical': 1, 'bipolar': 2, 'major depressive': 3, 'postpartum': 4, 'psychotic': 5}
id2label: {0: 'no depression', 1: 'atypical', 2: 'bipolar', 3: 'major depressive', 4: 'postpartum', 5: 'psychotic'}


In [ ]:
label2id

{'no depression': 0,
 'atypical': 1,
 'bipolar': 2,
 'major depressive': 3,
 'postpartum': 4,
 'psychotic': 5}

In [ ]:
id2label

{0: 'no depression',
 1: 'atypical',
 2: 'bipolar',
 3: 'major depressive',
 4: 'postpartum',
 5: 'psychotic'}

In [ ]:
df_org["labels"]=df_org.Labels.map(lambda x: label2id[x.strip()])

In [ ]:
df_org.head(5)

,Tweets,Labels,labels
6003,"i have a healthy head of hair, boyish good loo...",atypical,1
9482,dunlap mil sack amp lawson mil sacks he isn pl...,no depression,0
7625,still laughing abt dude saying nvm being silly...,no depression,0
4692,@mma_like I think you might have the Kanye bip...,bipolar,2
8894,because of the eras if he played vs people suc...,no depression,0


<Axes: ylabel='count'>

In [ ]:
df_org.Labels.value_counts().plot(kind='pie', figsize=(10,10))

In [ ]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import RobertaForSequenceClassification
model = RobertaForSequenceClassification.from_pretrained(
    "paulagarciaserrano/roberta-depression-detection",
    num_labels=NUM_LABELS,  # Make sure NUM_LABELS is set to the correct number of labels
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True  # Add this to handle size mismatches
)
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at paulagarciaserrano/roberta-depression-detection and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([6]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
SIZE= df_org.shape[0]

train_texts= list(df_org.Tweets[:SIZE//2])

val_texts=   list(df_org.Tweets[SIZE//2:(3*SIZE)//4 ])

test_texts=  list(df_org.Tweets[(3*SIZE)//4:])

train_labels= list(df_org.labels[:SIZE//2])

val_labels=   list(df_org.labels[SIZE//2:(3*SIZE)//4])

test_labels=  list(df_org.labels[(3*SIZE)//4:])

In [ ]:
len(train_texts)

7491

In [ ]:
len(train_texts), len(val_texts), len(test_texts)

(7491, 3746, 3746)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings  = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
class DataLoader(Dataset):
    """
    Custom Dataset class for handling tokenized text data and corresponding labels.
    Inherits from torch.utils.data.Dataset.
    """
    def __init__(self, encodings, labels):
        """
        Initializes the DataLoader class with encodings and labels.

        Args:
            encodings (dict): A dictionary containing tokenized input text data
                              (e.g., 'input_ids', 'token_type_ids', 'attention_mask').
            labels (list): A list of integer labels for the input text data.
        """
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        """
        Returns a dictionary containing tokenized data and the corresponding label for a given index.

        Args:
            idx (int): The index of the data item to retrieve.

        Returns:
            item (dict): A dictionary containing the tokenized data and the corresponding label.
        """
        # Retrieve tokenized data for the given index
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Add the label for the given index to the item dictionary
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        """
        Returns the number of data items in the dataset.

        Returns:
            (int): The number of data items in the dataset.
        """
        return len(self.labels)

In [ ]:
train_dataloader = DataLoader(train_encodings, train_labels)

val_dataloader = DataLoader(val_encodings, val_labels)

test_dataset = DataLoader(test_encodings, test_labels)

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    """
    Computes accuracy, F1, precision, and recall for a given set of predictions.

    Args:
        pred (obj): An object containing label_ids and predictions attributes.
            - label_ids (array-like): A 1D array of true class labels.
            - predictions (array-like): A 2D array where each row represents
              an observation, and each column represents the probability of
              that observation belonging to a certain class.

    Returns:
        dict: A dictionary containing the following metrics:
            - Accuracy (float): The proportion of correctly classified instances.
            - F1 (float): The macro F1 score, which is the harmonic mean of precision
              and recall. Macro averaging calculates the metric independently for
              each class and then takes the average.
            - Precision (float): The macro precision, which is the number of true
              positives divided by the sum of true positives and false positives.
            - Recall (float): The macro recall, which is the number of true positives
              divided by the sum of true positives and false negatives.
    """
    # Extract true labels from the input object
    labels = pred.label_ids

    # Obtain predicted class labels by finding the column index with the maximum probability
    preds = pred.predictions.argmax(-1)

    # Compute macro precision, recall, and F1 score using sklearn's precision_recall_fscore_support function
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')

    # Calculate the accuracy score using sklearn's accuracy_score function
    acc = accuracy_score(labels, preds)

    # Return the computed metrics as a dictionary
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

In [ ]:
!pip install accelerate==0.21.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install transformers[torch]

In [ ]:

!pip install accelerate

!pip install transformers[torch]

In [ ]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir='./TTC4900Model',
    do_train=True,
    do_eval=True,
    #  The number of epochs, defaults to 3.0
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    # Number of steps used for a linear warmup
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
   # TensorBoard log directory
    logging_dir='./multi-class-logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    fp16=True,
    load_best_model_at_end=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer, TrainingArguments
trainer = Trainer(
    # the pre-trained model that will be fine-tuned
    model=model,
     # training arguments that we defined above
    args=training_args,
    train_dataset=train_dataloader,
    eval_dataset=val_dataloader,
    compute_metrics= compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,1.632000,1.242014,0.607848,0.585800,0.652246,0.588766
100,0.819400,0.486681,0.842766,0.848732,0.875319,0.841843
150,0.463000,0.359884,0.873732,0.883710,0.906371,0.872419
200,0.331800,0.351350,0.878003,0.888128,0.920251,0.876207
250,0.352400,0.298505,0.900427,0.906111,0.913319,0.901952
300,0.295900,0.287542,0.895355,0.902863,0.911926,0.902051
350,0.261000,0.298880,0.905499,0.911507,0.910433,0.913805
400,0.278800,0.255652,0.914309,0.919985,0.925670,0.918215
450,0.241200,0.243424,0.917245,0.923399,0.928618,0.923039
500,0.216900,0.263842,0.910571,0.915910,0.921463,0.911844


TrainOutput(global_step=1407, training_loss=0.2664887942612044, metrics={'train_runtime': 727.458, 'train_samples_per_second': 30.893, 'train_steps_per_second': 1.934, 'total_flos': 1755453671823456.0, 'train_loss': 0.2664887942612044, 'epoch': 3.0})

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

In [ ]:
q=[trainer.evaluate(eval_dataset=df_org) for df_org in [train_dataloader, val_dataloader, test_dataset]]

pd.DataFrame(q, index=["train","val","test"]).iloc[:,:5]

,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.076631,0.976372,0.977956,0.977967,0.978169
val,0.257676,0.930326,0.934856,0.934748,0.935572
test,0.246941,0.927122,0.930018,0.928743,0.931861


from matplotlib import pyplot as plt
_df_12['eval_loss'].plot(kind='hist', bins=20, title='eval_loss')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_13['eval_Accuracy'].plot(kind='hist', bins=20, title='eval_Accuracy')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_14['eval_F1'].plot(kind='hist', bins=20, title='eval_F1')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_15['eval_Precision'].plot(kind='hist', bins=20, title='eval_Precision')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_16.plot(kind='scatter', x='eval_loss', y='eval_Accuracy', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_17.plot(kind='scatter', x='eval_Accuracy', y='eval_F1', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_18.plot(kind='scatter', x='eval_F1', y='eval_Precision', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_19.plot(kind='scatter', x='eval_Precision', y='eval_Recall', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_20['eval_loss'].plot(kind='line', figsize=(8, 4), title='eval_loss')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_21['eval_Accuracy'].plot(kind='line', figsize=(8, 4), title='eval_Accuracy')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_22['eval_F1'].plot(kind='line', figsize=(8, 4), title='eval_F1')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_23['eval_Precision'].plot(kind='line', figsize=(8, 4), title='eval_Precision')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
def predict(text):
    """
    Predicts the class label for a given input text

    Args:
        text (str): The input text for which the class label needs to be predicted.

    Returns:
        probs (torch.Tensor): Class probabilities for the input text.
        pred_label_idx (torch.Tensor): The index of the predicted class label.
        pred_label (str): The predicted class label.
    """
    # Tokenize the input text and move tensors to the GPU if available
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")

    # Get model output (logits)
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)
    """ Explanation outputs: The BERT model returns a tuple containing the output logits (and possibly other elements depending on the model configuration). In this case, the output logits are the first element in the tuple, which is why we access it using outputs[0].

    outputs[0]: This is a tensor containing the raw output logits for each class. The shape of the tensor is (batch_size, num_classes) where batch_size is the number of input samples (in this case, 1, as we are predicting for a single input text) and num_classes is the number of target classes.

    softmax(1): The softmax function is applied along dimension 1 (the class dimension) to convert the raw logits into class probabilities. Softmax normalizes the logits so that they sum to 1, making them interpretable as probabilities. """

    # Get the index of the class with the highest probability
    # argmax() finds the index of the maximum value in the tensor along a specified dimension.
    # By default, if no dimension is specified, it returns the index of the maximum value in the flattened tensor.
    pred_label_idx = probs.argmax()

    # Now map the predicted class index to the actual class label
    # Since pred_label_idx is a tensor containing a single value (the predicted class index),
    # the .item() method is used to extract the value as a scalar
    pred_label = model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label

In [ ]:
text = "some people didn't like mania and that's fine ig however (un)fortunately I have bipolar disorder and impeccable taste so I know for a fact that it fucks severely"
predict(text)

(tensor([[5.3668e-04, 1.9676e-04, 9.9870e-01, 1.3457e-04, 1.9792e-04, 2.3572e-04]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(2, device='cuda:0'),
 'bipolar')

In [ ]:
model_path = "depression-text-classification-model"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('depression-text-classification-model/tokenizer_config.json',
 'depression-text-classification-model/special_tokens_map.json',
 'depression-text-classification-model/vocab.json',
 'depression-text-classification-model/merges.txt',
 'depression-text-classification-model/added_tokens.json')

In [ ]:
model_path = "depression-text-classification-model"

# Load the model correctly
model = RobertaForSequenceClassification.from_pretrained(model_path)

# Load the tokenizer correctly, it should be a RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_path)

# Initialize the pipeline
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Now you can use nlp for predictions
nlp("My mind feels like a jumbled mess. I can't focus on anything for more than a few minutes, and even then, my thoughts are all over the place. It's like my brain is working against me. #brainfog #lackoffocus")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'atypical', 'score': 0.9977526068687439}]

In [ ]:
nlp("My mind feels like a jumbled mess. I can't focus on anything for more than a few minutes, and even then, my thoughts are all over the place. It's like my brain is working against me. #brainfog #lackoffocus")

[{'label': 'atypical', 'score': 0.9977526068687439}]

In [ ]:
nlp("Some days it feels like I'm living in a world of chaos, and it's hard to find any stability. It's like I'm living in a constant storm.")

[{'label': 'bipolar', 'score': 0.9982751607894897}]

In [ ]:
nlp("After having my baby, I find myself feeling so high one day and then so low the next. It’s a lot to navigate.")

[{'label': 'postpartum', 'score': 0.9991394281387329}]

In [ ]:
nlp("The love I have for my child is immense, but the weight of exhaustion sometimes feels heavier than anything.")

[{'label': 'postpartum', 'score': 0.9987815022468567}]